In [1]:
# !pip install transformers
!pip install nona

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/df-salary/train_rev1.csv


In [3]:
import torch, gc, random
from transformers.file_utils import is_tf_available, is_torch_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# %load_ext memory_profiler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import re
from nona.nona import nona

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 13 20:58:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
df = pd.read_csv('../input/df-salary/train_rev1.csv')
df = df.sample(200000,random_state=42)

In [7]:
df.drop('ContractType',axis=1,inplace=True)

def enc_with_na(df, column):
    enc = OneHotEncoder(sparse=False, drop='first')
    enc.fit(df[[column]].dropna())
    
    return enc,pd.DataFrame(enc.transform(df[[column]].dropna(
    )), index=df.loc[df.loc[:, column].notna()].index, columns=enc.get_feature_names())

In [8]:
ct_enc,ct_features = enc_with_na(df,'ContractTime')
cat_enc,cat_features = enc_with_na(df,'Category')

df = df.drop(['ContractTime','Category'],axis=1).join([ct_features,
cat_features])

features = df.drop(['Id','Title','FullDescription','LocationRaw','LocationNormalized','Company','SalaryRaw',
                   'SalaryNormalized','SourceName'],axis=1)

nona(data=features, algreg=make_pipeline(StandardScaler(with_mean=False), Ridge(
    alpha=0.1)), algclass=RandomForestClassifier(max_depth=2, random_state=42))

df = df.join(pd.DataFrame(ct_enc.inverse_transform(pd.DataFrame(
    features.loc[:, 'x0_permanent'])), index=df.index, columns=['ContractTime'])).drop(ct_enc.get_feature_names(), axis=1)
df = df.join(pd.DataFrame(cat_enc.inverse_transform(pd.DataFrame(
    features.drop('x0_permanent',axis=1))), index=df.index, columns=['Category'])).drop(cat_enc.get_feature_names(), axis=1)

df["FullDescription"] = df["FullDescription"].str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "")

df.FullDescription = 'Contract time for this vacancy is '+ df.ContractTime+'. ' + df.FullDescription
df.FullDescription = 'The category for this job is '+ df.Category+'. ' + df.FullDescription

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 29/29 [00:00<00:00, 1196.26it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_f

In [9]:
max_length = 128

In [10]:

# Make data
X = df.FullDescription
y = np.log1p(df.SalaryNormalized+1)


# Split Data
X_train, X_valid, y_train, y_valid = train_test_split(X.tolist(), y, test_size=.1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.1)

# Call the Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Encode the text
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(X_valid, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



class MakeTorchData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = MakeTorchData(train_encodings, y_train.ravel())
valid_dataset = MakeTorchData(valid_encodings, y_valid.ravel())
test_dataset = MakeTorchData(test_encodings, y_test.ravel())

In [11]:

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                           num_labels = 1).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

In [13]:
num_epochs = 5

In [14]:
os.environ["WANDB_DISABLED"] = "true"

In [15]:

# Specifiy the arguments for the trainer  
training_args = TrainingArguments(
    output_dir ='D:/',          
    num_train_epochs = num_epochs,     
    per_device_train_batch_size = 32,   
    per_device_eval_batch_size = 16,   
    weight_decay = 0.01,               
    learning_rate = 2e-5,
    logging_dir = './logs',            
    save_total_limit = 10,
    load_best_model_at_end = True,     
    metric_for_best_model = 'rmse',    
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    optim="adamw_torch"
) 

# Call the Trainer
trainer = Trainer(
    model = model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = valid_dataset,          
    compute_metrics = compute_metrics_for_regression,     
)

# Train the model
trainer.train()

# Call the summary
trainer.evaluate()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 162000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 25315


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,0.093000,0.083277,0.083277,0.288577,0.220120,0.653671,2.129631
2,0.077200,0.062945,0.062945,0.250888,0.184665,0.738227,1.789336
3,0.063400,0.058853,0.058853,0.242597,0.174391,0.755243,1.686801
4,0.055100,0.070401,0.070401,0.265331,0.202370,0.707220,1.959186
5,0.051000,0.059627,0.059627,0.244186,0.177942,0.752026,1.720433


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
Saving model checkpoint to D:/checkpoint-5063
Configuration saved in D:/checkpoint-5063/config.json
Model weights saved in D:/checkpoint-5063/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
Saving model checkpoint to D:/checkpoint-10126
Configuration saved in D:/checkpoint-10126/config.json
Model weights saved in D:/checkpoint-10126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
Saving model checkpoint to D:/checkpoint-15189
Configuration saved in D:/checkpoint-15189/config.json
Model weights saved in D:/checkpoint-15189/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch size = 16
Saving model checkpoint to D:/checkpoint-20252
Configuration saved in D:/checkpoint-20252/config.json
Model weights saved in D:/checkpoint-20252/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 20000
  Batch s

{'eval_loss': 0.08327678591012955,
 'eval_mse': 0.08327678591012955,
 'eval_rmse': 0.2885771691799164,
 'eval_mae': 0.2201199233531952,
 'eval_r2': 0.6536712649745798,
 'eval_smape': 2.1296306640625002,
 'eval_runtime': 78.1274,
 'eval_samples_per_second': 255.992,
 'eval_steps_per_second': 16.0,
 'epoch': 5.0}

In [16]:
pred = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 18000
  Batch size = 16


In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
mean_squared_error(np.exp(y_test),np.exp(pd.DataFrame(pred[0])))**.5

10715.423624403107